APRENDIZAJE DEL DETECTOR DE INTRUSIONES

El software para detectar intrusiones en la red protege una red informática de usuarios no autorizados, incluidos, quizás, personas con información privilegiada. La tarea de aprendizaje del detector de intrusiones es construir un modelo predictivo (es decir, un clasificador) capaz de distinguir entre conexiones "malas", llamadas intrusiones o ataques, y conexiones normales "buenas".

El Programa de evaluación de detección de intrusiones de DARPA de 1998 fue preparado y administrado por MIT Lincoln Labs. El objetivo era relevar y evaluar la investigación en detección de intrusos. Se proporcionó un conjunto estándar de datos para auditar, que incluye una amplia variedad de intrusiones simuladas en un entorno de red militar. El concurso de detección de intrusos KDD de 1999 utiliza una versión de este conjunto de datos.

Lincoln Labs estableció un entorno para adquirir nueve semanas de datos de volcado de TCP sin procesar para una red de área local (LAN) que simula una LAN típica de la Fuerza Aérea de EE. UU. Operaron la LAN como si fuera un verdadero entorno de la Fuerza Aérea, pero la salpicaron con múltiples ataques.

Los datos de entrenamiento sin procesar eran aproximadamente cuatro gigabytes de datos de volcado TCP binarios comprimidos de siete semanas de tráfico de red. Esto se procesó en aproximadamente cinco millones de registros de conexión. Del mismo modo, las dos semanas de datos de prueba arrojaron alrededor de dos millones de registros de conexión.

Una conexión es una secuencia de paquetes TCP que comienzan y terminan en momentos bien definidos, entre los cuales los datos fluyen hacia y desde una dirección IP de origen a una dirección IP de destino bajo algún protocolo bien definido. Cada conexión se etiqueta como normal o como un ataque, con exactamente un tipo de ataque específico. Cada registro de conexión consta de unos 100 bytes.

Los ataques se dividen en cuatro categorías principales:

    DOS: denegación de servicio, por ejemplo, syn flood;
    R2L: acceso no autorizado desde una máquina remota, por ejemplo, adivinar una contraseña;
    U2R: acceso no autorizado a privilegios de superusuario local (root), por ejemplo, varios ataques de "desbordamiento de búfer";
    Sondeo: vigilancia y otros sondeos, por ejemplo, escaneo de puertos.
    
Fuente: https://kdd.ics.uci.edu/databases/kddcup99/task.html

In [5]:
import numpy as np
import pandas as pd
import math as math
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
from sklearn.datasets import fetch_kddcup99

In [7]:
X, y = fetch_kddcup99(percent10=True, download_if_missing=True, return_X_y=True)

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:997)>

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
dt = [('duration', int),
      ('protocol_type', 'S4'),
      ('service', 'S11'),
      ('flag', 'S6'),
      ('src_bytes', int),
      ('dst_bytes', int),
      ('land', int),
      ('wrong_fragment', int),
      ('urgent', int),
      ('hot', int),
      ('num_failed_logins', int),
      ('logged_in', int),
      ('num_compromised', int),
      ('root_shell', int),
      ('su_attempted', int),
      ('num_root', int),
      ('num_file_creations', int),
      ('num_shells', int),
      ('num_access_files', int),
      ('num_outbound_cmds', int),
      ('is_host_login', int),
      ('is_guest_login', int),
      ('count', int),
      ('srv_count', int),
      ('serror_rate', float),
      ('srv_serror_rate', float),
      ('rerror_rate', float),
      ('srv_rerror_rate', float),
      ('same_srv_rate', float),
      ('diff_srv_rate', float),
      ('srv_diff_host_rate', float),
      ('dst_host_count', int),
      ('dst_host_srv_count', int),
      ('dst_host_same_srv_rate', float),
      ('dst_host_diff_srv_rate', float),
      ('dst_host_same_src_port_rate', float),
      ('dst_host_srv_diff_host_rate', float),
      ('dst_host_serror_rate', float),
      ('dst_host_srv_serror_rate', float),
      ('dst_host_rerror_rate', float),
      ('dst_host_srv_rerror_rate', float),
      ('labels', 'S16')]

In [ ]:
column_names = [c[0] for c in dt]
df_conexiones = pd.DataFrame(data=np.column_stack((X,y)),columns=column_names)

In [ ]:
# df_conexiones = df_conexiones.astype(dt)
df_conexiones = df_conexiones.infer_objects() # inferir un mejor tipo de datos

In [ ]:
df_conexiones.info()

In [ ]:
df_conexiones.head()

In [ ]:
df_conexiones.labels.unique()

**Ejercicio 1**

CONSIGNA:

1. Mediante la función SelectKBest de ScikitLearn, detectar los features más relevantes del dataset provisto

In [ ]:
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest, chi2
le = preprocessing.LabelEncoder()

In [ ]:

le.fit(df_conexiones['protocol_type'])
print(le.classes_)
# encoding protocol_type
df_conexiones['EncodedProtocolType'] = le.transform(
                                        df_conexiones['protocol_type'])
df_conexiones.EncodedProtocolType.unique()

In [ ]:
# encoding service
df_conexiones['EncodedService'] = le.fit_transform(
                                    df_conexiones['service'])
df_conexiones.EncodedService.unique()

In [ ]:
# encoding flag
df_conexiones['EncodedFlag'] = le.fit_transform(
                               df_conexiones['flag'])
df_conexiones.EncodedFlag.unique()

> remove that *b* before every value in labels

In [ ]:
df_conexiones.labels.unique()

In [ ]:
di = {b'normal.': "normal", 
      b'buffer_overflow.': "buffer_overflow", 
      b'loadmodule.': "loadmodule", 
      b'perl.': "perl",
      b'neptune.': "neptune", 
      b'smurf.': "smurf", 
      b'guess_passwd.': "guess_passwd", 
      b'pod.': "pod", 
      b'teardrop.': "teardrop",
      b'portsweep.': "portsweep", 
      b'ipsweep.': "ipsweep", 
      b'land.': "land", 
      b'ftp_write.': "ftp_write", 
      b'back.': "back",
      b'imap.': "imap", 
      b'satan.': "satan", 
      b'phf.': "phf",
      b'nmap.': "nmap" ,
      b'multihop.': "multihop",
      b'warezmaster.': "warezmaster", 
      b'warezclient.': "warezclient", 
      b'spy.': "spy", 
      b'rootkit.': "rootkit"}

In [ ]:
df_conexiones.replace({'labels':di}, inplace=True)

In [ ]:
df_conexiones.labels.unique()

> acaso este paso sobra?

In [ ]:
df_conexiones.loc[df_conexiones['labels'] == 'normal', 'EncodedLabels'] = 1
df_conexiones.loc[df_conexiones['labels'] != 'normal', 'EncodedLabels'] = 0

In [ ]:
df_conexiones.EncodedLabels.value_counts()

In [ ]:
df_conexiones['EncodedLabels2'] = le.fit_transform(df_conexiones['labels'])

In [ ]:
le.classes_

In [ ]:
df_corr = df_conexiones.corr()
df_corr

In [ ]:
# cols = df_corr.loc['EncodedLabels'].sort_values(ascending=True).index
# vals = df_corr.loc['EncodedLabels'].sort_values(ascending=True).values

# tam = len(vals)
# i = 0
# while i < tam:
#     if not (vals[i] is None) and (cols[i] != 'EncodedLabels'):
#         ax = plt.axes()
#         sns.scatterplot(x='EncodedLabels', 
#                           y=cols[i], 
#                           data=df_conexiones[['EncodedLabels',cols[i]]])
#         titulo = str(i) + ') Columna: ' + str(cols[i]) + '-/- Correlacion: ' + str(vals[i])
#         ax.set_title(titulo)
#         plt.show()
#         i += 1

In [ ]:
cols = df_corr.loc['EncodedLabels'].sort_values(ascending=True).index
vals = df_corr.loc['EncodedLabels'].sort_values(ascending=True).values
corrT = []

for i, e in enumerate(vals):
    if not (vals[i] is None) and (cols[i] != 'EncodedLabels'):
        corrT.append((cols[i], e))

corrT

In [ ]:
# cols = df_corr.loc['EncodedLabels2'].sort_values(ascending=True).index
# vals = df_corr.loc['EncodedLabels2'].sort_values(ascending=True).values
# corrT2 = []

# for i, e in enumerate(vals):
#     if not (vals[i] is None) and (cols[i] != 'EncodedLabels2'):
#         corrT2.append((cols[i], e))

# corrT2

In [ ]:
corrT.sort(key = lambda x: x[-1])
corrT
# corrT2.sort(key = lambda x: x[-1])
# corrT2

corrT

In [ ]:
corrT = pd.DataFrame(corrT)
corrT.dropna(inplace=True)
corrT.drop([4], inplace=True)
corrT

In [ ]:
print(corrT.head(5)) 
print(corrT.tail(5))

In [ ]:
df_conexiones_corr = df_conexiones[['count',
                                   'dst_host_count',
                                   'srv_count',
                                   'dst_host_same_src_port_rate',
                                   'dst_host_srv_serror_rate',
                                   'dst_host_srv_diff_host_rate',
                                   'same_srv_rate',
                                   'srv_diff_host_rate',
                                   'EncodedProtocolType',
                                   'logged_in',
                                   'EncodedLabels']]
df_conexiones_corr

In [ ]:
df_conexiones_corr = df_conexiones_corr.head(10000)

In [ ]:
df_conexiones_corr.shape

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import mutual_info_classif

In [ ]:
selector = SelectKBest(mutual_info_regression, k=4)

In [ ]:
X = df_conexiones_corr.drop(columns=['EncodedLabels'])

In [ ]:
y = df_conexiones_corr['EncodedLabels']

In [ ]:
selector.fit(X, y)

In [ ]:
scores = selector.scores_
list(X.columns)

In [ ]:
plt.rcParams['figure.figsize'] = [30,10]
plt.plot(scores)
plt.xticks(np.arange(10), list(X.columns))

**Ejercicio 2**

CONSIGNA:

1. Usando las variables más relevantes y separando los datos en "train" y "test", utilizar un árbol de decisión para realizar la predicción sobre si es una conexión normal o no:

In [1]:
X = df_conexiones[['count',
                  'srv_count',
                  'logged_in',
                  'EncodedProtocolType']]
y = df_conexiones[['EncodedLabels']]

NameError: name 'df_conexiones' is not defined

CONSIGNA:

1. Usando las variables más relevantes y separando los datos en "train" y "test", utilizar un árbol de decisión para realizar la predicción sobre que tipo de conexión es pero teniendo en cuenta todas las posibles:

**Ejercicio 3**

CONSIGNA:

1. Con el modelo del ejercicio 3, realizar esta vez una validación cruzada.
2. Mostrar la curva de validación

3. Aplicar GridSearch